# heatmap radial average vs pp_delay

In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np  
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import os
import src.pyabel_polar
import scipy

#### Utility functions

In [3]:
def generate_hdf_file_name(run_number):
    try:
        hdf_file_path = '/gpfs/bl1/current/processed/timepix_hdf/'
        file_start = "run_"+str(run_number).zfill(4)
        not_file_end = 'rawOnly.hdf5'
        hdf_file = [i for i in os.listdir(hdf_file_path) if os.path.isfile(os.path.join(hdf_file_path,i)) and i.startswith(file_start) and not i.endswith(not_file_end)][0]
        hdf_file_complete_path = hdf_file_path+hdf_file
        assert os.path.isfile(hdf_file_complete_path), 'File does not exist!'
        return hdf_file_complete_path
    except IndexError:
        print("Run",run_number,"do not exist!")

def number_of_trains_from_hdf(hdf_file_complete_path):
    '''Retrun number of recorded FEL trains in HDF file'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        trains = len(h_file['tpx3Times/triggerNr'][:])
    return trains

def data_from_hdf(hdf_file_complete_path, event_type = 'raw'):
    '''Read data from TimePix HDF files
    Choose raw or centroided data, default is centroided data'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        tof = h_file[str(event_type)+'/tof'][:]
        x_pos = h_file[str(event_type)+'/x'][:]
        y_pos = h_file[str(event_type)+'/y'][:]
    print('Reading from {} - number of events: {:.2e}'.format(hdf_file_complete_path, len(tof)))
    number_of_trains_from_hdf(hdf_file_complete_path)
    return tof, x_pos, y_pos

def data_sliced_by_tof(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, event_type = 'raw'):
    '''Slice data with respect to time-of-flight dimension'''
    tof, x_pos, y_pos = data_from_hdf(hdf_file_complete_path, event_type)
    sliced_x_pos = x_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_y_pos = y_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_tof = tof[np.logical_and(tof > tof_start, tof < tof_end)]
#     print('Slicing - number of events: {:.2e} | {:.2%}'.format(len(sliced_tof), len(sliced_tof)/len(tof)))
    return sliced_tof, sliced_x_pos, sliced_y_pos

def reduce_raw_data(tof, x_pos, y_pos, number_of_events):
    '''Reduce data for visualization'''
    return tof[:number_of_events], x_pos[:number_of_events],y_pos[:number_of_events]

def tof_conversion(tof, time_unit):
    '''Convert time axis'''
    if time_unit == None:
        return tof, 's'
    if time_unit == 'milli':
        return tof*10**3, 'ms'
    if time_unit == 'micro':
        return tof*10**6, 'us'

def plot_tof(tof, hist_bins=100, time_unit = None):     
    '''Plot time-of-flight spectrum via histogram'''
    fig = plt.subplots(num = 1)
    plt.clf()
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    plt.hist(tof, bins = hist_bins)
    plt.title('histogram: time-of-flight')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('number of events')
    plt.show()
    
def vmi_image(x_pos, y_pos, show_image = True):
    '''Display VMI image - cmax empirically found to surpress hot pixel '''
    fig = plt.figure(num = 6)
    plt.clf()
    counts, xbins, ybins, image = plt.hist2d(x_pos, y_pos,bins=np.linspace(0, 256, 257)) #, cmax= 1000)
    plt.colorbar()
    plt.xlabel('x_pos [px]')
    plt.ylabel('y_pos [px]')
    if show_image == False:
        plt.close()
    return counts
    
def display_tof_and_vmi_of_tof_interval(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, hist_bins = 100, time_unit = None, event_type = 'raw'):
    '''Display VMI Image and time-of-flight spetrum'''
    tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start, tof_end, event_type)
    plot_tof(tof, hist_bins, time_unit)
    vmi_image(x_pos,y_pos)

    
def transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius, non_plot = False):
    
    counts = vmi_image(x_pos, y_pos, show_image = False)
    image_cart = np.flipud(counts.transpose())
    
    image_polar, r_grid, theta_grid = src.pyabel_polar.reproject_image_into_polar(image_cart, origin=(x_center,y_center))
    radial_ave = np.sum(image_polar, axis=1)
    
    if non_plot == False:
        fig = plt.figure(figsize = (10,10),num = 6)
        plt.clf()
        plt.imshow(image_cart)
        plt.scatter(x_center, y_center, color='r')
        plt.gcf().gca().add_artist(plt.Circle((x_center, y_center), radius, color='r', fill=False))
        plt.xlabel('x_posr [px]')
        plt.ylabel('y_posr [px]')
        plt.title('VMI image')

    #     fig = plt.figure(num = 7)
    #     plt.clf()
    #     plt.imshow(image_polar)
    #     plt.title('Image - polar coordinates')

        fig = plt.figure(num = 8)
        plt.clf()
        plt.plot(radial_ave,'r-')
        plt.title('Radial average')
        plt.xlabel('r [px]')
        plt.ylabel('counts')
        plt.show()
    
    return radial_ave

def get_delay_stage_pos_from_txt_file(run_interval):
    runs = []
    delay_pos = []
    filepath = '/home/bl1user/Desktop/erk20919/CAMP/beamtime/run_pp-delay.txt'
    with open(filepath) as fp:
        for cnt, line in enumerate(fp):
            tt = [x.strip() for x in line.split(',')]
            runs.append(int(tt[0]))
            delay_pos.append(float(tt[1]))
    delay_pos = delay_pos[runs.index(run_interval[0]):runs.index(run_interval[1])+1]
    runs = runs[runs.index(run_interval[0]):runs.index(run_interval[1])+1]
    return runs, delay_pos

def save_all_delay_for_fragment(file_prefix, fragment ,run_interval, tof_start,tof_end, x_center, y_center, radius, timepix_event_type):
    runs_file , delay_stage = get_delay_stage_pos_from_txt_file(run_interval)
    for i in range(run_interval[0], run_interval[1]+1):
        print(delay_stage[i-run_interval[0]])
        hdf_file = generate_hdf_file_name(i)
        tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)
        radial_average = transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius, non_plot = True) 
        number_of_trains = number_of_trains_from_hdf(hdf_file)
        
        save_to_path = 'intermediate_data/'
        save_file_name = str(file_prefix)+"_"+str(fragment)+"_"+str(delay_stage[i-run_interval[0]])
        np.savez(save_to_path+save_file_name, radial_average=radial_average, fragment= fragment, delay_stage = delay_stage[i-run_interval[0]], number_of_trains = number_of_trains)


### Define source file & describe measurment

In [9]:
run_number = 185
timepix_event_type = 'raw'

hdf_file = generate_hdf_file_name(run_number)

### Complete time-of-flight

In [5]:
tof_start = 0E-6
tof_end = 12E-6
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)

plot_tof(tof, hist_bins = 500, time_unit = 'micro')

Reading from /gpfs/bl1/current/processed/timepix_hdf/run_0185_20191213-1441.hdf5 - number of events: 4.43e+07


### Slice time-of-flight dimension and create VMI image

In [6]:
# I2+ 
# tof_start, tof_end = 7.60E-6, 8.01E-6

# I3+ 
# tof_start, tof_end = 6.4E-6, 6.7E-6

# I4+ 
# tof_start, tof_end = 5.75E-6, 5.95E-6

# I5+ 
# tof_start, tof_end = 5.2E-6, 5.4E-6

# I6+ 
tof_start, tof_end = 4.85E-6, 5.02E-6

display_tof_and_vmi_of_tof_interval(hdf_file, tof_start, tof_end, event_type = timepix_event_type, time_unit = 'micro')

Reading from /gpfs/bl1/current/processed/timepix_hdf/run_0185_20191213-1441.hdf5 - number of events: 4.43e+07


### Chose center and calc radial average

In [7]:
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file, tof_start , tof_end, event_type = timepix_event_type)

Reading from /gpfs/bl1/current/processed/timepix_hdf/run_0185_20191213-1441.hdf5 - number of events: 4.43e+07


In [8]:
# fragment = 'I2+'
# x_center, y_center = 134, 118        
# radius = 80

# fragment = 'I3+'       #  -> no clear peak 
# x_center, y_center = 131, 127          
# radius = 18

# fragment = 'I4+'
# x_center, y_center = 131, 127          
# radius = 18

# fragment = 'I5+'
# x_center, y_center = 131, 127          
# radius = 18

fragment = 'I6+'
x_center, y_center = 133, 119        
radius = 35

radial_average = transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius)

### Save to intermediate results 

In [32]:
file_prefix = '800nm_delay'
run_interval = [208, 209]

In [33]:
save_all_delay_for_fragment(file_prefix, fragment ,run_interval, tof_start, tof_end, x_center, y_center, radius, timepix_event_type)

320.0
Reading from /gpfs/bl1/current/processed/timepix_hdf/run_0208_20191213-1901.hdf5 - number of events: 4.98e+07
0.0
Reading from /gpfs/bl1/current/processed/timepix_hdf/run_0209_20191213-1913.hdf5 - number of events: 1.81e+07


___
# Create heatmap

In [34]:
def simple_plots(radial_averages_norm, delay_stage_pos, fragment):
    colors = cm.rainbow(np.linspace(0, 1, len(radial_averages_norm)))
    fig = plt.figure(figsize = (10,10),num = 10)
    for i in range(len(radial_averages_norm)):
        plt.plot(radial_averages_norm[i],label=delay_stage_pos[i], color = colors[i])
    plt.xlabel('r')
    plt.ylabel('counts')
    plt.legend()
    plt.title(str(fragment), fontsize = 20)
    print('Number of delay steps:',len(delay_stage_pos))
    plt.show()
    
def heatmap(radial_averages_norm, fragment):
    fig = plt.figure(figsize = (10,10),num = 11)
    image = np.column_stack(radial_averages_norm)
    plt.imshow(image, aspect="auto", origin="lower")
    plt.xticks(np.arange(len(delay_stage_pos)), delay_stage_pos)
    plt.title(str(fragment), fontsize = 20)
    print('Number of delay steps:',len(delay_stage_pos))
    plt.show()
    
    
def load_radial_data(file_prefix,fragment):
    
    load_from_path = 'intermediate_data/'
    files = [i for i in os.listdir(load_from_path) if os.path.isfile(os.path.join(load_from_path,i)) and str(file_prefix)+"_"+str(fragment) in i]
    
    radial_averages =[]
    delay_stage_pos = []
    norm_counts = []
    
    for i in files:
        npzfile = np.load(load_from_path+i)
        radial_averages.append(npzfile['radial_average'])
        delay_stage_pos.append(float(npzfile['delay_stage']))
        fragment = (npzfile['fragment'])
        norm_counts.append(float(npzfile['number_of_trains']))
        
    radial_averages = np.array(radial_averages)
    delay_stage_pos = np.array(delay_stage_pos)
    norm_counts = np.array(norm_counts)

    sort_ind = np.argsort(delay_stage_pos)
    radial_averages = radial_averages[sort_ind]
    norm_counts = norm_counts[sort_ind]
    delay_stage_pos = delay_stage_pos[sort_ind]
    
    radial_averages_norm = []
    for i in range(len(radial_averages)):
        radial_averages_norm.append(radial_averages[i]/norm_counts[i])
    radial_averages_norm = np.array(radial_averages_norm)

    print('fragment:',fragment) 
    print('number of delay_stage_pos:',len(delay_stage_pos))
    print('delay_stage_steps:',delay_stage_pos)

    return radial_averages_norm, delay_stage_pos     
    

### Re-read data

In [35]:
file_prefix = '800nm_delay'
fragment = 'I6+'

radial_averages_norm, delay_stage_pos = load_radial_data(file_prefix,fragment)

fragment: I6+
number of delay_stage_pos: 24
delay_stage_steps: [  0.   320.   330.   335.   340.   341.   342.   343.   344.   344.25
 344.5  344.75 345.   345.25 345.5  345.75 345.82 346.   346.25 346.5
 346.75 347.   347.5  348.  ]


### simple plots

In [30]:
simple_plots(radial_averages_norm, delay_stage_pos, fragment)

Number of delay steps: 22


### heatmap

In [37]:
heatmap(radial_averages_norm, fragment)

Number of delay steps: 24
